In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
#from genomic_benchmarks.seq2loc import fasta2loc
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.notebook import tqdm
from pathlib import Path
import yaml
import gzip
from Bio import SeqIO

def _fastagz2dict(fasta_path, fasta_total=None, stop_id=None, region_name_transform=lambda x: x):
    # load gzipped fasta into dictionary
    fasta = {}

    with gzip.open(fasta_path, "rt") as handle:
        for record in tqdm(SeqIO.parse(handle, "fasta"), total=fasta_total):
            fasta[region_name_transform(record.id)] = str(record.seq)
            if stop_id and (record.id == stop_id):
                # stop, do not read small contigs
                break
    return fasta

In [17]:
from tqdm.autonotebook import tqdm as tdm

def fasta2loc(fasta_path, ref_dict, use_seq_ids=True):

    tree = {}
    nseqs = 0

    # building tree for seq searching
    for seq in SeqIO.parse(open(fasta_path, "r"), "fasta"):
        s = str(seq.seq)
        rev = str(seq.seq.reverse_complement())
        if use_seq_ids:
            sname = seq.name
        else:
            sname = s
        nseqs += 1

        _update_tree(tree, s, sname, "+")
        _update_tree(tree, rev, sname, "-")

    print(f"{nseqs} sequences read and parsed.")

    results = {}

    for chrom in tdm(ref_dict):
        curr_positions = []
        # print(f"Processing chrom {chrom}.")

        for i, c in tdm(enumerate(ref_dict[chrom]), total=len(ref_dict[chrom]), leave=False):

            prev_positions = curr_positions + [tree]
            curr_positions = []

            for pos in prev_positions:
                if c in pos:
                    pos = pos[c]
                    curr_positions.append(pos)

                    if "terminal" in pos:
                        results[pos["terminal"][0]] = (chrom, i - pos["terminal"][2] + 1, i + 1, pos["terminal"][1])

    print(f"{len(results.keys())} sequences found in the reference.")

    return results


def _update_tree(root, seq_str, seq_name, direction):
    # updates tree in `root` with a sequence `seq_str`
    position = root

    for c in seq_str:
        if c in position:
            position = position[c]
        else:
            position[c] = {}
            position = position[c]
    position["terminal"] = (seq_name, direction, len(seq_str))

## Load genomic reference and download data from GitHub

In [18]:
genome = _fastagz2dict("./Athaliana_167_TAIR9.fa.gz",
                      7)
genome.keys()

  0%|          | 0/7 [00:00<?, ?it/s]

dict_keys(['Chr1', 'Chr2', 'Chr3', 'Chr4', 'Chr5', 'ChrM', 'ChrC'])

## Get coding sequences

In [19]:
# slow!
import shutil
with gzip.open('./Athaliana_167_TAIR10.cds.fa.gz', 'rb') as f_in:
  with open('./Athaliana_167_TAIR10.cds.fa', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

In [20]:
coding_seqs = fasta2loc("./Athaliana_167_TAIR10.cds.fa", genome)

35386 sequences read and parsed.


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/30427671 [00:00<?, ?it/s]

  0%|          | 0/19698289 [00:00<?, ?it/s]

  0%|          | 0/23459830 [00:00<?, ?it/s]

  0%|          | 0/18585056 [00:00<?, ?it/s]

  0%|          | 0/26975502 [00:00<?, ?it/s]

  0%|          | 0/366924 [00:00<?, ?it/s]

  0%|          | 0/154478 [00:00<?, ?it/s]

6954 sequences found in the reference.


### A few checks

In [22]:
len(coding_seqs.keys())

6954

In [23]:
import itertools
#print 3 randomly selected keys
test_keys = list(itertools.islice(coding_seqs, 3))
print(test_keys)
print(dict(list(coding_seqs.items())[0: 5]))

['AT1G01030.1', 'AT1G01073.1', 'AT1G01115.1']
{'AT1G01030.1': ('Chr1', 11863, 12940, '-'), 'AT1G01073.1': ('Chr1', 44676, 44787, '+'), 'AT1G01115.1': ('Chr1', 56623, 56740, '+'), 'AT1G01120.1': ('Chr1', 57391, 58978, '-'), 'AT1G01180.1': ('Chr1', 75632, 76556, '+')}


In [9]:
coding_seqs[test_keys[1]]

('Chr1', 44676, 44787, '+')

In [10]:
genome['Chr1'][44676:44787]

'ATGCTTATCAGTATCAGCCCACTGATATTTGTGATACCAGTATCATCTGACGTGGCTTCTTCTGATTGGTTACATTTGACAAAAGCAAAAAATATTATATATATTTATTAA'

In [23]:
coding_seqs['AT1G01115.1']

('Chr1', 56623, 56740, '+')

In [9]:
from Bio.Seq import Seq

def _rev(seq, strand):
    # reverse complement
    if strand == '-':
        return str(Seq(seq).reverse_complement())
    else:
        return seq

In [10]:
_rev(genome['Chr1'][44676:44787], "+")

'ATGCTTATCAGTATCAGCCCACTGATATTTGTGATACCAGTATCATCTGACGTGGCTTCTTCTGATTGGTTACATTTGACAAAAGCAAAAAATATTATATATATTTATTAA'

In [12]:
cds_df = pd.DataFrame.from_dict(coding_seqs, orient='index', columns=['region','start','end','strand'])

cds_df = cds_df.rename_axis('id')
cds_df.to_csv("positive.csv")
cds_df.head(10)

,region,start,end,strand
id,,,,
AT1G01030.1,Chr1,11863,12940,-
AT1G01073.1,Chr1,44676,44787,+
AT1G01115.1,Chr1,56623,56740,+
AT1G01120.1,Chr1,57391,58978,-
AT1G01180.1,Chr1,75632,76556,+
AT1G01240.3,Chr1,100682,101678,+
AT1G01250.1,Chr1,104730,105309,-
AT1G01260.3,Chr1,109594,111367,+
AT1G01300.1,Chr1,117064,118522,+


## Get non-coding sequences

In [37]:
import numpy as np
def get_chr_names_and_lengths():
    chr_lengths = {}
    for chromosome in genome.keys():
        chr_lengths[chromosome] = len(genome[chromosome])

    # check that all lengths are different from 0
    assert all(x != 0 for x in chr_lengths.values())

    return chr_lengths

def get_random_chr(chr_names_and_lengths):
    chr_lengths = pd.Series(chr_names_and_lengths.values())
    chr_probs = chr_lengths / chr_lengths.sum()
    chr_names = list(chr_names_and_lengths.keys())
    return chr_names[np.argwhere(np.random.multinomial(1, chr_probs))[0][0]]


def is_intersecting(c, pos, df_forbidden):
    intersecting = (df_forbidden['region'] == c) & (df_forbidden['start'].astype(int) <= pos) & (
                df_forbidden['end'].astype(int) >= pos)
    return intersecting.any()


def get_random_pos(df_forbidden: pd.DataFrame, chr_names_and_lengths, offset_from_end):
    c = get_random_chr(chr_names_and_lengths)
    c_len = chr_names_and_lengths[c]
    pos = np.random.randint(c_len - offset_from_end) + 1

    while is_intersecting(c, pos, df_forbidden):
        pos = np.random.randint(c_len) + 1

    return c, pos

chr_names_and_lengths = get_chr_names_and_lengths()
print(chr_names_and_lengths)
get_random_pos(cds_df, chr_names_and_lengths, 0)

{'Chr1': 30427671, 'Chr2': 19698289, 'Chr3': 23459830, 'Chr4': 18585056, 'Chr5': 26975502, 'ChrM': 366924, 'ChrC': 154478}


('Chr2', 8921741)

In [45]:
num_seqs = len(cds_df)
negative_samples = [None] * num_seqs
for i in range(num_seqs):
    while True:
        seq_length = cds_df.iloc[i]['end'] - cds_df.iloc[i]['start']
        chrom, start = get_random_pos(cds_df, chr_names_and_lengths, seq_length)
        end = start + seq_length
        seq = genome[chrom][start:end]
        if 'N' not in seq.upper():
            negative_samples[i] = [chrom, start, end, '+']
            break
neg_df = pd.DataFrame(negative_samples, columns=['region', 'start', 'end', 'strand'])
neg_df

,region,start,end,strand
0,Chr3,20153559,20154636,+
1,Chr1,20758206,20758317,+
2,Chr1,16583551,16583668,+
3,Chr2,694237,695824,+
4,Chr5,14792508,14793432,+
...,...,...,...,...
6949,Chr3,19258295,19258826,+
6950,ChrM,97823,98342,+
6951,Chr1,29821352,29822534,+
6952,Chr1,26327564,26327831,+


In [55]:
neg_df.index.name = "id"
neg_df.to_csv("negative.csv", index=True)

## Train/test split

In [59]:
train_cds, test_cds = train_test_split(cds_df, shuffle=True, random_state=42)
train_cds.shape, test_cds.shape

((5215, 4), (1739, 4))

In [56]:
train_neg_seqs, test_neg_seqs = train_test_split(neg_df, shuffle=True, random_state=42)
train_neg_seqs.shape, test_neg_seqs.shape

((5215, 4), (1739, 4))

## CSV files

In [ ]:
BASE_FILE_PATH = Path('../../datasets/demo_arabidopsis_coding_vs_intergenomic_seqs/')
(BASE_FILE_PATH / 'train').mkdir()
(BASE_FILE_PATH / 'test').mkdir()
train_cds.to_csv(BASE_FILE_PATH / 'train' / 'positive.csv.gz', index=False, compression='gzip')
train_neg_seqs.to_csv(BASE_FILE_PATH / 'train' / 'negative.csv.gz', index=False, compression='gzip')
test_cds.to_csv(BASE_FILE_PATH / 'test' / 'positive.csv.gz', index=False, compression='gzip')
test_neg_seqs.to_csv(BASE_FILE_PATH / 'test' / 'negative.csv.gz', index=False, compression='gzip')

## Create YAML File

In [64]:
with open(BASE_FILE_PATH / 'metadata.yaml', 'w') as fw:
    desc = {
        'version': 0,
        'classes': {
            'positive': {
                'type': 'fa.gz',
                'url': 'https://raw.githubusercontent.com/framazan/files/master/Athaliana_167_TAIR10.cds.fa.gz',
                'extra_processing': 'PHYTOZOME_ARABIDOPSIS_V10' 
            },    
            'negative': {
                'type': 'fa.gz',
                'url': 'https://raw.githubusercontent.com/framazan/files/master/Athaliana_167_TAIR9.fa.gz',
                'extra_processing': 'PHYTOZOME_ARABIDOPSIS_V10'
            }
        }
    }
    
    yaml.dump(desc, fw)

desc

{'version': 0,
 'classes': {'positive': {'type': 'fa.gz',
   'url': 'https://raw.githubusercontent.com/framazan/files/master/Athaliana_167_TAIR10.cds.fa.gz',
   'extra_processing': 'PHYTOZOME_ARABIDOPSIS_V10'},
  'negative': {'type': 'fa.gz',
   'url': 'https://raw.githubusercontent.com/framazan/files/master/Athaliana_167_TAIR9.fa.gz',
   'extra_processing': 'PHYTOZOME_ARABIDOPSIS_V10'}}}

## Cleaning

In [ ]:
!rm positive.csv negative.csv